In [2]:
%load_ext autoreload
%autoreload 2
from collections import OrderedDict
from datasets import load_dataset
from more_itertools import chunked
from itda import ITDAConfig, ITDA
from tqdm.auto import tqdm
import numpy, torch
import json, os
os.makedirs("dataset", exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16, device_map="balanced")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
torch.set_grad_enabled(False)

In [5]:
import numpy as np
nf4 = np.asarray(
    [
        -1.0,
        -0.6961928009986877,
        -0.5250730514526367,
        -0.39491748809814453,
        -0.28444138169288635,
        -0.18477343022823334,
        -0.09105003625154495,
        0.0,
        0.07958029955625534,
        0.16093020141124725,
        0.24611230194568634,
        0.33791524171829224,
        0.44070982933044434,
        0.5626170039176941,
        0.7229568362236023,
        1.0,
    ]
)
image_max = 6.0

In [6]:
prompts_dataset = load_dataset("opendiffusionai/cc12m-cleaned")
prompts_iterator = prompts_dataset["train"]["caption_llava_short"]
guidance_scale = 3.5
num_inference_steps = 1
batch_size = 32
width = 256
height = 256
d_model = 3072
itda_config = ITDAConfig(
    d_model=d_model,
    target_l0=64,
    loss_threshold=0.6,
    add_error=True,
    fvu_loss=True,
    subtract_mean=False,
)

In [7]:
len(prompts_iterator)

8531902

In [8]:
normalize = True
device = torch.device("cuda:0")
itda = ITDA(itda_config, dtype=torch.bfloat16, device=device)

In [9]:

import os
os.environ["JAX_PLATFORMS"] = "cpu"
import jax
from pathlib import Path
import orbax.checkpoint as ocp
import shutil
from loguru import logger


def restore_array(path):
    with jax.default_device(jax.devices("cpu")[0]):
        checkpointer = ocp.PyTreeCheckpointer()
        vals_meta = checkpointer.metadata(path)
        sharding = jax.sharding.SingleDeviceSharding(jax.devices("cpu")[0])
        restore_args = ocp.checkpoint_utils.construct_restore_args(
            vals_meta,
            sharding_tree=jax.tree.map(lambda _: sharding, vals_meta)
        )
        restored_vals = checkpointer.restore(path, restore_args=restore_args)
    return restored_vals

In [12]:
from huggingface_hub import HfApi, hf_hub_download
from tqdm.auto import tqdm
import os


def download_subdirectory(repo_id, subdir, local_dir, repo_type="model"):
    print(f"Downloading {subdir} from {repo_id} to {local_dir}")
    # Initialize HfApi
    api = HfApi()
    
    # List all files in the repository
    print(f"Listing files in {repo_id}")
    all_files = api.list_repo_files(repo_id=repo_id, repo_type=repo_type)
    
    # Filter files that belong to the target subdirectory
    target_files = [f for f in all_files if f.startswith(f"{subdir}/")]
    
    print(f"Found {len(target_files)} files in {subdir}")
    # Create local directory if it doesn't exist
    os.makedirs(local_dir, exist_ok=True)
    
    # Download each file individually
    for file in tqdm(target_files):
        print(f"Downloading {file}")
        hf_hub_download(
            repo_id=repo_id,
            filename=file,
            repo_type=repo_type,
            local_dir=local_dir,
            local_dir_use_symlinks=False  # Save actual files instead of symlinks
        )
    print(f"Downloaded {len(target_files)} files to {local_dir}")

In [17]:
dpath = download_subdirectory("nev/flux1-saes", "sae_double_l18_img", "flux1-saes")

Listing files in nev/flux1-saes
Found 15 files in sae_double_l18_img


  0%|          | 0/15 [00:00<?, ?it/s]

/home/nebius/flux-saes/flux_saes/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


_CHECKPOINT_METADATA:   0%|          | 0.00/262 [00:00<?, ?B/s]

_METADATA:   0%|          | 0.00/9.45k [00:00<?, ?B/s]

...

76dbfbe41aa969d5990b0f105649e8f3:   0%|          | 0.00/443k [00:00<?, ?B/s]

a3c2e057ba1c099136ec478d0e308055:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

e6ecbf024b232ab1bb30b028ef1b1a7c:   0%|          | 0.00/182 [00:00<?, ?B/s]

manifest.ocdbt:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloaded 15 files to flux1-saes


In [47]:
layer = 18
sae_mid = restore_array(os.path.abspath(
    f"flux1-saes/sae_double_l{layer}_img/30000/default"))

In [48]:
sae_mid["info"].keys()

fs_means = sae_mid["info"]["feature_means"]
fs_sq_means = sae_mid["info"]["feature_square_means"]

In [49]:
stds = np.sqrt(np.maximum(fs_sq_means - np.square(fs_means), 1e-6))

In [50]:
wm = sae_mid["info"]["whitening_matrix"]

In [51]:
w_dec = (sae_mid["sae_params"]["W_dec"]  * stds) @ wm.T

w_dec = torch.tensor(w_dec)

In [ ]:
itda.xs.data = w_dec
itda.ys.data = w_dec

itda.dictionary_size = w_dec.shape[0]

itda.mean_x.data = torch.tensor(fs_means @ wm.T)
itda.mean_y.data = torch.tensor(fs_means @ wm.T)

In [53]:
from caching_utils import save_itda_outputs
from scored_storage import ScoredStorage
from pathlib import Path

feature_acts = ScoredStorage(
    Path("itda_data_saes") / "feature_acts.db",
    3, 1024
)

In [55]:
images_dir = Path("images_itda_saes")
image_activations_dir = Path("image_activations_itda_saes")

In [56]:
for i, prompts in enumerate(chunked((bar := tqdm(prompts_iterator[10000:11000])), batch_size)):
    with torch.inference_mode():
        for m in pipe.transformer.modules():
            m._forward_hooks = OrderedDict()
        text_outputs = {}
        image_outputs = {}
        timestep = 0
        def save_hook(self, input, output):
            text_outputs[timestep] = output[0]
            image_outputs[timestep] = output[1]
        pipe.transformer.transformer_blocks[18].register_forward_hook(save_hook)
        height = 256
        width = 256
        def callback_on_step_end(self, i, t, kwargs):
            global timestep
            timestep = i
            return {}
        pipe.set_progress_bar_config(disable=True)
        with torch.inference_mode():
            latents = pipe(
                prompts,
                height=height,
                width=width,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                max_sequence_length=512,
                generator=torch.Generator("cpu").manual_seed(0),
                return_dict=False,
                callback_on_step_end=callback_on_step_end,
                output_type="latent",
            )[0]
        latents_reshaped = pipe._unpack_latents(latents, height, width, pipe.vae_scale_factor)
        latents_to_be_compressed = latents_reshaped.cpu().float().numpy()
        
        x = image_outputs[0].to(torch.bfloat16).to(device)
        x = x.reshape(-1, d_model)

        out = itda(x, x)

        weights, indices = out.weights, out.indices

        # weights, indices = (x @ itda.xs[:itda.dictionary_size].T
        #                 ).topk(itda.config.target_l0, dim=-1)
        
        weights = weights.float().cpu().numpy()
        indices = indices.cpu().numpy()

        weights = weights.reshape(image_outputs[0].shape[0], image_outputs[0].shape[1], -1)
        indices = indices.reshape(image_outputs[0].shape[0], image_outputs[0].shape[1], -1)

        save_itda_outputs(
            weights, indices, prompts=prompts, images=latents_to_be_compressed, feature_acts=feature_acts, save_image_activations=True, step=i, 
            images_dir=images_dir, image_activations_dir=image_activations_dir
        )

  0%|          | 0/1000 [00:00<?, ?it/s]

AssertionError: 

In [ ]:
1/0

In [ ]:
itda.xs - itda_clone.xs

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [ ]:
weights.shape, indices.shape, latents_to_be_compressed.shape

NameError: name 'weights' is not defined

In [ ]:
from matplotlib import pyplot as plt
bins = 1000
with torch.inference_mode():
    xs = itda.xs[:itda.dictionary_size]
    pp = xs[torch.randperm(len(xs))[:100]] @ xs.T
    mn = pp.min().item()
    mx = pp.max().item()
    hist = torch.histc(pp, bins=bins,) / pp.numel()
# plt.plot(np.linspace(mn, mx, bins), hist.cpu().numpy())
plt.scatter(np.log2(np.abs(np.linspace(mn, mx, bins))), hist.cpu().numpy())
plt.yscale("log")
plt.show()

In [ ]:
# from tqdm.auto import trange
# @torch.compile#(mode="reduce-overhead")
# def step(xs):
#     prods = (torch.nn.functional.cosine_similarity(xs[:, None, :], xs[None, :, :], dim=-1) - torch.eye(len(xs), device=xs.device, dtype=xs.dtype)).flatten()
#     max_prod = prods.argmax().item()
#     max_a = max_prod // len(xs)
#     max_b = max_prod % len(xs)
#     combined = xs[max_a] + xs[max_b]
#     xs[max_a] = combined
#     xs[max_b], xs[-1] = xs[-1], xs[max_b]
#     xs = xs[:-1]
#     return xs, max_prod > 0.5
# with torch.inference_mode():
#     xs = itda.xs[:itda.dictionary_size].clone()
#     # for i in (bar := trange(5000)):
#     for i in trange(5000):
#         xs, stop = step(xs)
#         if stop:
#             break

In [ ]:
%%sh
uv sync
uv pip install \
    --extra-index-url=https://pypi.nvidia.com \
    "cudf-cu12==25.2.*" "cuml-cu12==25.2.*" --prerelease=allow --index-strategy unsafe-best-match

Audited 2 packages in 6ms


CPU times: user 24.2 s, sys: 213 ms, total: 24.4 s
Wall time: 24.5 s


KMeans()

tensor([[-0.0141,  0.0201, -0.0138,  ...,  0.0184,  0.0069,  0.0067],
        [-0.0096, -0.0100, -0.0234,  ...,  0.0041,  0.0097, -0.0040],
        [ 0.0420,  0.0256,  0.0227,  ...,  0.0082, -0.0022, -0.0042],
        ...,
        [ 0.0162,  0.0035, -0.0098,  ...,  0.0096,  0.0286,  0.0162],
        [-0.0085,  0.0152, -0.0327,  ..., -0.0248,  0.0037, -0.0085],
        [-0.0117, -0.0345, -0.0163,  ..., -0.0020,  0.0023,  0.0173]],
       device='cuda:0')